# Data preparation

In this notebook we preprocess our data for ML algorithms and create new features.

## Load and reshape data

Import libraries

In [13]:
!pip install tsfresh

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime  
from datetime import timedelta  
import gc
import pickle
from sklearn import preprocessing, metrics
import pandas as pd
import matplotlib.pyplot as plt
#from sagemaker import get_execution_role
#import boto3
import seaborn as sns
import itertools,operator
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, load_robot_execution_failures
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import EfficientFCParameters, MinimalFCParameters, ComprehensiveFCParameters
from sklearn.model_selection import train_test_split

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Helper functions

In [0]:
# this function is taken from https://www.kaggle.com/ragnar123/very-fst-model
def reduce_mem_usage(df, verbose=True):
    '''Reduce memory usage of dataframe by converting ints and floats 
    Args:
        df: dataframe
            
    Returns:
        dataframe with converted columns
    '''
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


Read data.

In [17]:
print('Reading files...')
#data_location = 's3://{}/{}/{}'.format(bucket, prefix, 'calendar.csv')
data_folder = '/content/drive/My Drive/Kaggle_M5/01_preprocessed_data/'
X = pd.read_pickle(data_folder + 'X.pkl')

Reading files...


In [18]:
X.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,weekday,month,event_name_1,event_type_1,year,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,start_date,days_from_start,start_date_from_start,sell_price
46881672,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,5,no event,no event,2016,no event,no event,0,0,0,2011-01-29,1940,0,2.980469
46881673,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,5,no event,no event,2016,no event,no event,0,0,0,2011-01-29,1940,0,2.480469
46881674,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,5,no event,no event,2016,no event,no event,0,0,0,2011-01-29,1940,0,3.980469
46881675,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,5,no event,no event,2016,no event,no event,0,0,0,2013-08-24,1940,938,1.280273
46881676,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,5,no event,no event,2016,no event,no event,0,0,0,2014-03-01,1940,1127,1.000000


## Lag features

In [19]:
lags = [x for x in range(28, 28+8)]
windows = [3, 7]

lag_cols = [f"lag_{lag}" for lag in lags ]
for lag, lag_col in zip(lags, lag_cols):
    print('Lag {}'.format(lag))
    X[lag_col] = X.groupby("id")["demand"].shift(lag)

X = reduce_mem_usage(X)
lags = [28]
for window in windows:
    for lag,lag_col in zip(lags, lag_cols):
        print("Lag {}, window {}".format(lag, window))
        X[f"rmean_{lag}_{window}"] = X[["id", lag_col]].groupby("id")[lag_col].\
        transform(lambda x : x.rolling(window).mean())
X = reduce_mem_usage(X)
# For each id drop early rows for which lag cannot be calculated 
    
#X = X.dropna()


Lag 28
Lag 29
Lag 30
Lag 31
Lag 32
Lag 33
Lag 34
Lag 35
Mem. usage decreased to 3444.25 Mb (43.8% reduction)
Lag 28, window 3
Lag 28, window 7
Mem. usage decreased to 3623.09 Mb (0.0% reduction)


In [20]:
X.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'demand', 'date', 'wm_yr_wk', 'weekday', 'month', 'event_name_1',
       'event_type_1', 'year', 'event_name_2', 'event_type_2', 'snap_CA',
       'snap_TX', 'snap_WI', 'start_date', 'days_from_start',
       'start_date_from_start', 'sell_price', 'lag_28', 'lag_29', 'lag_30',
       'lag_31', 'lag_32', 'lag_33', 'lag_34', 'lag_35', 'rmean_28_3',
       'rmean_28_7'],
      dtype='object')

In [0]:
X = X.dropna()

In [0]:
X.to_pickle(data_folder + 'X_lags.pkl')

In [0]:
#X = pd.read_pickle(data_folder + "X_lags.pkl")

# From train, validation, and test datasets

Split X into X_train and X_val

In [0]:
first_day = pd.Timestamp(X['date'].values.min())
last_day =  pd.Timestamp(X['date'].values.max())
n_val_days = 28# round(n_days*val_size)
n_test_days = 28# round(n_days*val_size)

first_val_day  =  last_day - timedelta(days = (n_test_days + n_val_days  -1 ))
first_test_day =  last_day - timedelta(days = n_test_days - 1)


In [25]:
print("first_val_day {}".format(first_val_day))
print("first_test_day {}".format(first_test_day))


first_val_day 2016-03-28 00:00:00
first_test_day 2016-04-25 00:00:00


In [26]:
X_train = X[X['date'] < first_val_day]
X_val = X[(X['date'] >= first_val_day) & (X['date'] < first_test_day)]
X_test = X[X['date'] >= first_test_day]
y_train = X_train['demand']
y_val = X_val['demand']


X_test = reduce_mem_usage(X_test)
X_train = reduce_mem_usage(X_train)
X_val = reduce_mem_usage(X_val)
X = reduce_mem_usage(X)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Mem. usage decreased to 66.72 Mb (1.2% reduction)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Mem. usage decreased to 3408.75 Mb (0.0% reduction)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Mem. usage decreased to 67.54 Mb (0.0% reduction)
Mem. usage decreased to 3540.65 Mb (0.0% reduction)


In [0]:
#X_train = X[X['date'] < first_val_day]
#X_val = X[(X['date'] >= first_val_day) & (X['date'] < first_test_day)]

#X =  X[X['date'] < first_test_day]
#y = X['demand']
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, 
                                                    #random_state=0)

In [0]:
del X

### Sales pattern

In this part we determine demand_type of the time series, as it was done at https://github.com/Mcompetitions/M5-methods

Determine average demand interval.

In [30]:
temp = X_train.groupby(['id'])['demand'].sum() 
temp[temp == 0]

id
FOODS_3_135_WI_2_validation        0.0
HOUSEHOLD_1_032_TX_1_validation    0.0
HOUSEHOLD_1_400_CA_4_validation    0.0
HOUSEHOLD_1_518_CA_2_validation    0.0
Name: demand, dtype: float64

In [0]:
X_train['ADI'] = X_train.groupby(['id'])['demand'].transform(lambda x:  0 if (sum(x) == 0) else len(x)/sum(x > 0))

Compute square of the Coefficient of Variation (CV²).

In [0]:
X_train['CV2'] = X_train.groupby(['id'])['demand'].transform(lambda x: (np.std(x[x>0])/np.mean(x[x>0]))**2)

Calculate demand_type.

In [0]:
X_train['demand_type'] = ((X_train['ADI'] <= 1.32) & (X_train['CV2'] < 0.5))*1 + \
    ((X_train['ADI'] > 1.32) & (X_train['CV2'] < 0.5))*10 + \
    ((X_train['ADI'] <= 1.32) & (X_train['CV2'] >= 0.5))*100 + \
    ((X_train['ADI'] > 1.32) & (X_train['CV2'] > 0.5))*1000 
X_train['demand_type'] = X_train['demand_type'].replace({1:'smooth', 10:'intermittent', 100:'erratic', 1000:'lumpy'})

In [0]:
c = 'demand_type'
col_type = X_train[c].dtype
if col_type == 'object' or col_type.name == 'category':
    X_train[c] = X_train[c].astype('category')


In [0]:
temp = X_train[['id', 'ADI', 'CV2', 'demand_type']].drop_duplicates()

In [0]:
X_val = X_val.merge(temp, on = ['id'], how = 'left')
X_test = X_test.merge(temp, on = ['id'], how = 'left')
del temp

### Aggregations

Add average sales per week per item, per month per item and simply average sales per item. Add maximum items sold.

In [0]:

X_train['avg_weekday_demand'] = X_train.groupby(['id', 'weekday'])['demand'].transform('mean') 
X_train['avg_demand'] =  X_train.groupby(['id'])['demand'].transform('mean') 
X_train['max_demand'] =  X_train.groupby(['id'])['demand'].transform('max') 
X_train['std_demand'] =  X_train.groupby(['id'])['demand'].transform('std') 
X_train['zero_seq_len'] = X_train.groupby(['id'])['demand'].transform(lambda x: 
                          max(len(list(y)) for (c,y) in itertools.groupby(x) if c==0))
X_train['non_zero_seq_len'] = X_train.groupby(['id'])['demand'].transform(lambda x: 
  0 if sum(x)== 0 else  max(len(list(y)) for (c,y) in itertools.groupby([1 if z>0 else 0 for z in x ]) if c > 0))


In [0]:
my_parameters = {
 'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
  {'f_agg': 'median', 'maxlag': 40},
  {'f_agg': 'var', 'maxlag': 40}],
  'ar_coefficient': [{'coeff': 3, 'k': 10}],
}

In [39]:
demand_ts = X_train[['item_id', 'date', 'demand']]
demand_ts_fe = extract_features(demand_ts, 
                     column_id='item_id', column_sort='date',
                     default_fc_parameters=my_parameters)

Feature Extraction: 100%|██████████| 10/10 [01:30<00:00,  9.06s/it]


In [0]:
demand_ts_fe = demand_ts_fe.reset_index()
demand_ts_fe = demand_ts_fe.rename(columns={"id": "item_id"})

In [0]:
X_train = X_train.merge(demand_ts_fe, on = ['item_id'], how = 'left')
X_val = X_val.merge(demand_ts_fe, on = ['item_id'], how = 'left')
X_test = X_test.merge(demand_ts_fe, on = ['item_id'], how = 'left')


In [0]:
X_train['quantile025_week'] = X_train.groupby(['id', 'weekday'])['demand'].transform(lambda x: x.quantile(0.25))

In [0]:
X_train['max_weekday_demand'] = X_train.groupby(['id', 'weekday'])['demand'].transform('max') 

Add average price per item

In [0]:
X_train['avg_price'] = X_train.groupby(['id'])['sell_price'].transform('mean') 

In [0]:
temp = X_train[['id', 'avg_price', 'avg_demand', 'max_demand', 'std_demand',
                'non_zero_seq_len', 'zero_seq_len']].drop_duplicates()

In [0]:
X_val = X_val.merge(temp, on = ['id'], how = 'left')
X_test = X_test.merge(temp, on = ['id'], how = 'left')


In [0]:
temp = X_train[['id', 'weekday', 'avg_weekday_demand', 'quantile025_week', 
                'max_weekday_demand']].drop_duplicates()

In [0]:
X_val = X_val.merge(temp, on = ['id', 'weekday'], how = 'left')
X_test = X_test.merge(temp, on = ['id', 'weekday'], how = 'left')


In [0]:
X_train['cheaper_than_usual'] = (X_train['sell_price'] < X_train['avg_price'])
X_val['cheaper_than_usual'] = (X_val['sell_price'] < X_val['avg_price'])
X_test['cheaper_than_usual'] = (X_test['sell_price'] < X_test['avg_price'])

In [0]:
X_train = X_train.drop(columns = ['wm_yr_wk'])
X_val = X_val.drop(columns = ['wm_yr_wk'])
X_test = X_test.drop(columns = ['wm_yr_wk'])

In [51]:
X_train = reduce_mem_usage(X_train)
X_val = reduce_mem_usage(X_val)
X_test = reduce_mem_usage(X_test)


Mem. usage decreased to 4670.67 Mb (35.1% reduction)
Mem. usage decreased to 91.96 Mb (34.7% reduction)
Mem. usage decreased to 91.15 Mb (34.9% reduction)


In [0]:
X_val.to_pickle(data_folder + "X_val.pkl")
X_test.to_pickle(data_folder + "X_test.pkl")
X_train.to_pickle(data_folder + "X_train.pkl")


In [0]:
#del X_test
#del X_val, y_val
#del X_train, y_train

In [54]:
X_val.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'demand', 'date', 'weekday', 'month', 'event_name_1', 'event_type_1',
       'year', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI',
       'start_date', 'days_from_start', 'start_date_from_start', 'sell_price',
       'lag_28', 'lag_29', 'lag_30', 'lag_31', 'lag_32', 'lag_33', 'lag_34',
       'lag_35', 'rmean_28_3', 'rmean_28_7', 'ADI', 'CV2', 'demand_type',
       'demand__agg_autocorrelation__f_agg_"mean"__maxlag_40',
       'demand__agg_autocorrelation__f_agg_"median"__maxlag_40',
       'demand__agg_autocorrelation__f_agg_"var"__maxlag_40',
       'demand__ar_coefficient__k_10__coeff_3', 'avg_price', 'avg_demand',
       'max_demand', 'std_demand', 'non_zero_seq_len', 'zero_seq_len',
       'avg_weekday_demand', 'quantile025_week', 'max_weekday_demand',
       'cheaper_than_usual'],
      dtype='object')

## Correlations with demand

In [55]:
X_train.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'demand', 'date', 'weekday', 'month', 'event_name_1', 'event_type_1',
       'year', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI',
       'start_date', 'days_from_start', 'start_date_from_start', 'sell_price',
       'lag_28', 'lag_29', 'lag_30', 'lag_31', 'lag_32', 'lag_33', 'lag_34',
       'lag_35', 'rmean_28_3', 'rmean_28_7', 'ADI', 'CV2', 'demand_type',
       'avg_weekday_demand', 'avg_demand', 'max_demand', 'std_demand',
       'zero_seq_len', 'non_zero_seq_len',
       'demand__agg_autocorrelation__f_agg_"mean"__maxlag_40',
       'demand__agg_autocorrelation__f_agg_"median"__maxlag_40',
       'demand__agg_autocorrelation__f_agg_"var"__maxlag_40',
       'demand__ar_coefficient__k_10__coeff_3', 'quantile025_week',
       'max_weekday_demand', 'avg_price', 'cheaper_than_usual'],
      dtype='object')

In [56]:
numerical_features = ['days_from_start', 'sell_price',
       'lag_28', 'lag_29', 'lag_30', 'lag_31', 'lag_32', 'lag_33', 'lag_34',
       'rmean_28_7', 'rmean_28_3',
       'ADI', 'CV2', 'avg_weekday_demand', 'avg_demand', 'max_demand',
       'std_demand', 'quantile025_week', 'max_weekday_demand', 'avg_price',
       'zero_seq_len', 'non_zero_seq_len',
       'demand__agg_autocorrelation__f_agg_"mean"__maxlag_40',
       'demand__agg_autocorrelation__f_agg_"median"__maxlag_40',
       'demand__agg_autocorrelation__f_agg_"var"__maxlag_40',
       'demand__ar_coefficient__k_10__coeff_3']
for feature in numerical_features:
    print("{}: {}".format(feature, X_train[feature].corr(X_train['demand'])))

days_from_start: -0.03996247651158752
sell_price: -0.1513283127030617
lag_28: 0.668002419750039
lag_29: 0.6527569517440468
lag_30: 0.634829192352799
lag_31: 0.6247735505052979
lag_32: 0.6209171779489195
lag_33: 0.6268593469844339
lag_34: 0.6413506181185805
rmean_28_7: 0.7322895290054409
rmean_28_3: 0.7144788129636607
ADI: -0.14116904656842705
CV2: 0.04408657325472014
avg_weekday_demand: 0.7378555158643114
avg_demand: 0.7247737532318004
max_demand: 0.5382724257803655
std_demand: 0.65335088072252
quantile025_week: 0.617229443522539
max_weekday_demand: 0.5922586124354491
avg_price: -0.15094942769833902
zero_seq_len: -0.075947569021157
non_zero_seq_len: 0.5363686690917953
demand__agg_autocorrelation__f_agg_"mean"__maxlag_40: 0.1752657104177005
demand__agg_autocorrelation__f_agg_"median"__maxlag_40: 0.13178857869099725
demand__agg_autocorrelation__f_agg_"var"__maxlag_40: 0.40743117245958743
demand__ar_coefficient__k_10__coeff_3: 0.09785724569566116


### Save by demand type

In [57]:
X_train = X_train.dropna()
for i in ['smooth', 'erratic', 'lumpy', 'intermittent']:
    print(i)
    X_val[X_val['demand_type'] == i].to_pickle(data_folder + "X_val_{}.pkl".format(i))
    X_train[X_train['demand_type'] == i].to_pickle(data_folder + "X_train_{}.pkl".format(i))
    X_test[X_test['demand_type'] == i].to_pickle(data_folder + "X_test_{}.pkl".format(i))
    print()

smooth

erratic

lumpy

intermittent



In [58]:
X_test.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'demand', 'date', 'weekday', 'month', 'event_name_1', 'event_type_1',
       'year', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI',
       'start_date', 'days_from_start', 'start_date_from_start', 'sell_price',
       'lag_28', 'lag_29', 'lag_30', 'lag_31', 'lag_32', 'lag_33', 'lag_34',
       'lag_35', 'rmean_28_3', 'rmean_28_7', 'ADI', 'CV2', 'demand_type',
       'demand__agg_autocorrelation__f_agg_"mean"__maxlag_40',
       'demand__agg_autocorrelation__f_agg_"median"__maxlag_40',
       'demand__agg_autocorrelation__f_agg_"var"__maxlag_40',
       'demand__ar_coefficient__k_10__coeff_3', 'avg_price', 'avg_demand',
       'max_demand', 'std_demand', 'non_zero_seq_len', 'zero_seq_len',
       'avg_weekday_demand', 'quantile025_week', 'max_weekday_demand',
       'cheaper_than_usual'],
      dtype='object')

## Save by store_id

In [0]:
stores = X_test['store_id'].values.unique()

In [60]:
stores

[CA_1, CA_2, CA_3, CA_4, TX_1, TX_2, TX_3, WI_1, WI_2, WI_3]
Categories (10, object): [CA_1, CA_2, CA_3, CA_4, ..., TX_3, WI_1, WI_2, WI_3]

In [61]:

for i in stores:
    print(i)
    X_val[X_val['store_id'] == i].to_pickle(data_folder + "X_val_{}.pkl".format(i))
    X_train[X_train['store_id'] == i].to_pickle(data_folder + "X_train_{}.pkl".format(i))
    X_test[X_test['store_id'] == i].to_pickle(data_folder + "X_test_{}.pkl".format(i))
    print()

CA_1

CA_2

CA_3

CA_4

TX_1

TX_2

TX_3

WI_1

WI_2

WI_3



In [62]:
X_val.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'demand', 'date', 'weekday', 'month', 'event_name_1', 'event_type_1',
       'year', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI',
       'start_date', 'days_from_start', 'start_date_from_start', 'sell_price',
       'lag_28', 'lag_29', 'lag_30', 'lag_31', 'lag_32', 'lag_33', 'lag_34',
       'lag_35', 'rmean_28_3', 'rmean_28_7', 'ADI', 'CV2', 'demand_type',
       'demand__agg_autocorrelation__f_agg_"mean"__maxlag_40',
       'demand__agg_autocorrelation__f_agg_"median"__maxlag_40',
       'demand__agg_autocorrelation__f_agg_"var"__maxlag_40',
       'demand__ar_coefficient__k_10__coeff_3', 'avg_price', 'avg_demand',
       'max_demand', 'std_demand', 'non_zero_seq_len', 'zero_seq_len',
       'avg_weekday_demand', 'quantile025_week', 'max_weekday_demand',
       'cheaper_than_usual'],
      dtype='object')